In [27]:
import sys, os
sys.path.insert(0, "/home/storage/hans/jax_reco_new")
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import jax.numpy as jnp
from jax.scipy import optimize
import jax
jax.config.update("jax_enable_x64", True)
import optimistix as optx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lib.simdata_i3 import I3SimBatchHandlerTFRecord
from lib.geo import center_track_pos_and_time_based_on_data_batched_v
from lib.experimental_methods import get_clean_pulses_fn_v
from lib.network import get_network_eval_v_fn

from likelihood_mpe_padded_input_biweight_conv_clip_charge import get_neg_c_triple_gamma_llh
from lib.geo import get_xyz_from_zenith_azimuth, __c
from dom_track_eval import get_eval_network_doms_and_track as get_eval_network_doms_and_track
import time
import glob

dtype = jnp.float32
eval_network_v = get_network_eval_v_fn(bpath='/home/storage/hans/jax_reco/data/network',
                                       dtype=dtype)
eval_network_doms_and_track = get_eval_network_doms_and_track(eval_network_v, dtype=dtype)

# Create padded batches (with different seq length).

tfrecord = "/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_*_to_*_1st_pulse.tfrecord"
fs = glob.glob(tfrecord)
#print(fs, len(fs)) # 3422
tfrecord = "/home/storage2/hans/i3files/21217/ftr/data_ds_21217_from_*_to_*_1st_pulse.tfrecord"
#fs = glob.glob(tfrecord)
#print(fs, len(fs)) # 198670
fs += glob.glob(tfrecord)

print(fs)
print(len(fs))

batch_maker = I3SimBatchHandlerTFRecord(fs, batch_size=128)
batch_iter = batch_maker.get_batch_iterator()

['/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_8000_to_9000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_6000_to_7000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_7000_to_8000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_0_to_1000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_4000_to_5000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_5000_to_6000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_3000_to_4000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_1000_to_2000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_2000_to_3000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21220/ftr/data_ds_21220_from_9000_to_10000_1st_pulse.tfrecord', '/home/storage2/hans/i3files/21217/ftr/data_ds_21217_from_22000_to_23000_1st_puls

In [28]:
n_events = 0
for i in range(100):
        data, mctruth = batch_iter.next() # [Nev, Ndom, Nobs], [Nev, Naux]
        print(f"processing batch with shape ({data.shape[0]}, {data.shape[1]}, {data.shape[2]})")
        n_events += data.shape[0]

processing batch with shape (19622, 44, 5)
processing batch with shape (26382, 32, 5)
processing batch with shape (35470, 24, 5)
processing batch with shape (14594, 60, 5)
processing batch with shape (23538, 17, 5)
processing batch with shape (17784, 24, 5)
processing batch with shape (19751, 32, 5)
processing batch with shape (14986, 44, 5)
processing batch with shape (3521, 60, 5)
processing batch with shape (9856, 81, 5)
processing batch with shape (6037, 109, 5)
processing batch with shape (4005, 147, 5)
processing batch with shape (2687, 198, 5)
processing batch with shape (1731, 266, 5)
processing batch with shape (1140, 359, 5)
processing batch with shape (627, 483, 5)
processing batch with shape (279, 650, 5)
processing batch with shape (76, 874, 5)
processing batch with shape (6, 1176, 5)


StopIteration: 

In [29]:
n_events

202092